In [1]:
import pandas as pd
print('Libaries imported')

Libaries imported


# Task 1

1) Read Dataframes from wikipedia html page and define the dataframe [0] as relevant data frame. Also rename column Postcode in Postalcode as shown in the example in the task.

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Mydata=df[0]
Mydata.rename(columns={"Postcode":"PostalCode"}, inplace=True)
Mydata.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Task 1
2) dropping all rows where the value in Borough is "not assigned"

In [3]:
Mydata1=Mydata.query('Borough != "Not assigned"').reset_index(drop=True)
Mydata1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### Task 1
3) grouping and combining neighbourhoods in same postalcodeareas

In [4]:
Mydata2 = Mydata1.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(list)
Mydata2 = Mydata2.sample(frac=1).reset_index()
Mydata2['Neighbourhood']= Mydata2['Neighbourhood'].str.join(', ')
Mydata2.head()

,PostalCode,Borough,Neighbourhood
0,M5E,Downtown Toronto,Berczy Park
1,M6J,West Toronto,"Little Portugal, Trinity"
2,M6R,West Toronto,"Parkdale, Roncesvalles"
3,M5M,North York,"Bedford Park, Lawrence Manor East"
4,M4K,East Toronto,"The Danforth West, Riverdale"


#### Task 1
4) replacing "not assigned" Neighboorhood with Borough value

In [5]:
Mydata2.loc[Mydata2['Neighbourhood'] == "Not assigned","Neighbourhood"] = Mydata2.loc[Mydata2['Neighbourhood'] == "Not assigned", "Borough"]

#### Task 1
5) shape method

In [6]:
Mydata2.shape

(103, 3)

# Task 2

1) Read csv with Coordinates

In [7]:
df = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

In [8]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Task 2
2) Merge Coordinates & Neighbourhood table

In [9]:
Geodata=Mydata2.merge(df, how='left', left_on='PostalCode', right_on='Postal Code')
Geodata.drop(['Postal Code'], axis=1, inplace=True)
Geodata.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
3,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188


In [10]:
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Task 3

1) Select only neighbourhoods in Downtown Toronto 

In [11]:
toronto_data = Geodata[Geodata['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
4,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
7,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049


#### Task 3
2) Visualize Map and Neighbourhoods

In [12]:
map_downtown = folium.Map(location=[43.655617, -79.379510], zoom_start=12)

for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  

map_downtown

#### Task 3
3) Load Venues in Neighbouhood via Foursquare API

In [13]:
CLIENT_ID = 'FXC5O32EF1UJ5TDLDZQB3Y3DMNUNP2LKKKJZK2TLQSJE3O25' 
CLIENT_SECRET = 'FGETIPOHOIUXSI04403QKSW2JG1ELCTTDF52XBT2HILHMJ3A'
VERSION = '20180605' 
LIMIT=100
radius=500

In [14]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Berczy Park
Cabbagetown, St. James Town
Design Exchange, Toronto Dominion Centre
Stn A PO Boxes 25 The Esplanade
Harbourfront East, Toronto Islands, Union Station
Central Bay Street
First Canadian Place, Underground city
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Christie
Harbord, University of Toronto
Church and Wellesley
Adelaide, King, Richmond
Chinatown, Grange Park, Kensington Market
Commerce Court, Victoria Hotel
St. James Town
Ryerson, Garden District
Harbourfront, Regent Park
Rosedale


In [15]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.040000,0.0,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.010000,0.0,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"Cabbagetown, St. James Town",0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.022222,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011364,0.0,...,0.0,0.0,0.0,0.0,0.011364,0.0,0.0,0.011364,0.0,0.011364


#### Task 3
4) Get the Top 10 venues in each Neighbourhood

In [16]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                  Bar  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04
5     Asian Restaurant  0.03
6           Steakhouse  0.03
7     Sushi Restaurant  0.03
8                Hotel  0.03
9           Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2            Beer Bar  0.03
3          Steakhouse  0.03
4  Seafood Restaurant  0.03
5         Cheese Shop  0.03
6      Farmers Market  0.03
7                Café  0.03
8              Bakery  0.03
9            Creperie  0.02


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4          Boutique  0.06
5               Bar  0.06
6  Sculpture Garden

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,American Restaurant,Bar,Sushi Restaurant,Asian Restaurant,Burger Joint,Hotel,Steakhouse
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Belgian Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Airport Gate,Airport Food Court
3,"Cabbagetown, St. James Town",Coffee Shop,Park,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Chinese Restaurant,Breakfast Spot
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Gym / Fitness Center,Bubble Tea Shop,Salad Place,Japanese Restaurant


#### Task 3
5) Cluster Neighborhoods in 4 clusters by using k-means

In [19]:
kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Bakery,Café,Cheese Shop,Steakhouse,Seafood Restaurant,Beer Bar,Farmers Market,Belgian Restaurant
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,3,Coffee Shop,Park,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Bakery,Chinese Restaurant,Breakfast Spot
2,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,3,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Italian Restaurant,Gastropub,Seafood Restaurant,Bar,Steakhouse
3,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,3,Coffee Shop,Café,Restaurant,Hotel,Cocktail Bar,Seafood Restaurant,Italian Restaurant,Beer Bar,Pub,Farmers Market
4,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,3,Coffee Shop,Aquarium,Hotel,Café,Brewery,Fried Chicken Joint,Italian Restaurant,Scenic Lookout,Restaurant,History Museum


#### Task 3
6) Map of Clusters

In [20]:

map_clusters = folium.Map(location=[43.655617, -79.379510], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters